In [2]:
from nga.utils.utils import (
    load_config, save_as_image, read_depth_map, compute_errors,
    load_metadata
)
from nerfstudio.utils.eval_utils import eval_load_checkpoint
from nerfstudio.cameras.rays import RayBundle
from nerfstudio.utils import colormaps
from PIL import Image
from pathlib import Path
import yaml
import torch
import json
import numpy as np
import matplotlib.pyplot as plt

In [3]:
config_path = "outputs/checkered_cube/nga-instant-ngp/2023-09-30_140921/config.yml"
config_path = "outputs/white_line/nga-nerfacto/2023-10-03_142931/config.yml"
# config_path = "outputs/checkered_sphere/nga-instant-ngp/2023-09-27_224714/config.yml"

config = load_config(config_path)
config.load_dir = config.get_checkpoint_dir()

meta, data_dir = load_metadata(config)


In [4]:

# setup pipeline (which includes the DataManager)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pipeline = config.pipeline.setup(device=device, test_mode="val")
pipeline.eval()


checkpoint_path, step = eval_load_checkpoint(config, pipeline)
results_path = config.get_base_dir() / "results.json"
render_output_path = config.get_base_dir() / "renders"
render_output_path.mkdir(parents=True, exist_ok=True)

[18:39:22] Auto image downscale factor of 1                                                 ]8;id=309028;file:///home/ccl/.conda/envs/nerfstudio/lib/python3.8/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py\nerfstudio_dataparser.py]8;;\:]8;id=516165;file:///home/ccl/.conda/envs/nerfstudio/lib/python3.8/site-packages/nerfstudio/data/dataparsers/nerfstudio_dataparser.py#349\349]8;;\

            Dataset is overriding train_indices to [10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,    ]8;id=205471;file:///home/ccl/Code/nerf-geometry-analysis/nga/data/dataparsers/nga_dataparser.py\nga_dataparser.py]8;;\:]8;id=152680;file:///home/ccl/Code/nerf-geometry-analysis/nga/data/dataparsers/nga_dataparser.py#159\159]8;;\
           21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42,                      
           43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64,                      
           65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86,                      
           87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106,                       
           107, 108, 109]                                                                                               

            Dataset is overriding val_indices to [110, 111, 112, 113, 114, 115, 116, 117, 118,     ]8;id=67019;file:///home/ccl/Code/nerf-geometry-analysis/nga/data/dataparsers/nga_dataparser.py\nga_dataparser.py]8;;\:]8;id=680839;file:///home/ccl/Code/nerf-geometry-analysis/nga/data/dataparsers/nga_dataparser.py#159\159]8;;\
           119]                                                                                                         

Setting up training dataset...

Caching all 100 images.

...

Setting up evaluation dataset...

Caching all 10 images.

Output()

Loading latest checkpoint from load_dir

✅ Done loading checkpoint from outputs/white_line/nga-nerfacto/2023-10-03_142931/nerfstudio_models/step-000024000.ckpt

In [5]:
import torch
from nerfstudio.cameras.rays import RayBundle
from nerfstudio.model_components.ray_samplers import SpacedSampler
import plotly.graph_objects as go
import plotly.express as px


In [20]:
import numpy as np
from nga.utils.spacial import convert_to_transformed_space
num_samples = 100
padding = 0
pad_scale = (num_samples + 2*padding) / num_samples

dataparser_scale = pipeline.datamanager.train_dataparser_outputs.dataparser_scale
dimensions = np.array([1, 0.1,0.1])*pad_scale
near = 0
far = 1 * pipeline.datamanager.train_dataparser_outputs.dataparser_scale * dimensions[2]
n = num_samples
y = torch.linspace(-0.5*dimensions[1], 0.5*dimensions[1], n+1)
y = (y[1:] + y[:-1]) / 2
x = torch.full_like(y, 0.1*dimensions[0])
z = torch.full_like(y, 0.5*dimensions[2])

# Initialize the UniformSampler
sampler = SpacedSampler(lambda x : x, lambda x : x, num_samples, train_stratified=False)
origins=torch.stack([x, y, z], dim=-1)
directions=torch.zeros_like(origins)
directions[..., 2] = -1
origins_trans = convert_to_transformed_space(origins, pipeline.datamanager.train_dataparser_outputs)
directions_trans = convert_to_transformed_space(directions, pipeline.datamanager.train_dataparser_outputs, is_direction=True)
ray_bundle = RayBundle(
    origins=origins_trans,
    directions=directions_trans,
    pixel_area=torch.full([n, 1], dimensions[1]*dimensions[2]/(n*n)),
    nears=torch.full([n, 1], near),
    fars=torch.full([n, 1], far),
    camera_indices=torch.zeros([n, 1]),
)

# Generate samples
samples = sampler.generate_ray_samples(ray_bundle).to("cuda")

densities = pipeline.model.get_densities(samples)
log_densities = np.log1p(densities.cpu().squeeze().detach().numpy())


depth = ((samples.frustums.starts + samples.frustums.ends) / 2).cpu() / dataparser_scale
pts = origins.view(n, 1, 3) + directions.view(n, 1, 3) * depth

fig = go.Figure(data=go.Heatmap(
    z=log_densities.transpose(),
    x=pts[:,0,1],
    y=pts[0,:,2],
    colorscale='Viridis')
)
fig.update_layout(
    title="Vertical Slice of Densities",
    xaxis_title="Y",
    yaxis_title="Z",
    width=500,
    height=500,
    hovermode=False,
    showlegend=False,
    margin=dict(l=20, r=20, t=50, b=20)
)
fig.show()

In [21]:
from nga.utils.spacial import convert_from_transformed_space

lcs_percentage = np.pi * 0.05**2
lcs_percentile = 100-100*lcs_percentage
print("lcs_percentile", lcs_percentile)
threshold = np.percentile(log_densities, lcs_percentile)
solid = log_densities >= threshold

solid_center = np.mean(pts[solid, :].numpy(), axis=0)

print(solid_center)

lcs_percentile 99.21460183660255
[ 0.09999993 -0.00294304 -0.00360126]


In [22]:

fig = go.Figure(data=go.Heatmap(
    z=np.array(solid, np.float32).transpose(),
    x=pts[:,0,1],
    y=pts[0,:,2],
    colorscale='Viridis')
)
fig.add_trace(go.Scatter(
    x=[solid_center[1]],
    y=[solid_center[2]],
    mode='markers',
    marker=dict(
        color='red',
        size=3
    )
))
fig.update_layout(
    title="Solid Region",
    xaxis_title="Y",
    yaxis_title="Z",
    width=500,
    height=500,
    hovermode=False,
    showlegend=False,
    margin=dict(l=20, r=20, t=50, b=20)
)
fig.show()

In [26]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import numpy as np

# Create a subplot layout
fig = make_subplots(rows=1, cols=2,
                    subplot_titles=("Vertical Slice of Densities", "Solid Region"),
                    horizontal_spacing=0.15,
)

# Add the first heatmap to subplot (1, 1)
trace1 = go.Heatmap(
    z=log_densities.transpose(),
    x=pts[:, 0, 1],
    y=pts[0, :, 2],
    colorscale='Viridis',
    colorbar=dict(
        x=0.425,  # Position of colorbar for first subplot
        title="Log Density"
    )
)
fig.add_trace(trace1, row=1, col=1)

# Add the second heatmap to subplot (1, 2)
trace2 = go.Heatmap(
    z=np.array(solid, np.float32).transpose(),
    x=pts[:, 0, 1],
    y=pts[0, :, 2],
    colorscale='Viridis',
    colorbar=dict(
        x=1.0,  # Position of colorbar for second subplot
        title="Solid Region"
    )
)
fig.add_trace(trace2, row=1, col=2)

# Add a scatter plot on top of the second heatmap
trace3 = go.Scatter(
    x=[solid_center[1]],
    y=[solid_center[2]],
    mode='markers',
    marker=dict(
        color='red',
        size=3
    )
)
fig.add_trace(trace3, row=1, col=2)

# Update layout
fig.update_layout(
    xaxis_title="Y",
    yaxis_title="Z",
    xaxis2_title="Y",
    yaxis2_title="Z",
    width=800,
    height=400,
    hovermode=False,
    showlegend=False,
    margin=dict(l=20, r=20, t=50, b=20)
)

# Show figure
fig.show()


: 

In [10]:
fig.write_image("figure.svg")
fig.write_image("figure.jpeg")